# Seismic visualization notebook

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import os
import imageio
warnings.filterwarnings("ignore")

from segysak.segy import (
    segy_loader,
    get_segy_texthead,
    segy_header_scan,
    segy_header_scrape,
    well_known_byte_locs,
)

#### Import seismic volume and horizon

In [7]:
path_data = './data/'

# Load seismic data
cube = segy_loader(path_data+'volve10r12-full-twt-sub3d.sgy', **well_known_byte_locs('petrel_3d'))

# Calculate corner points of seismic survey
cube.seis.calc_corner_points()
corners = np.array(cube.attrs["corner_points_xy"])

# Load horizon
hrz = pd.read_csv(path_data+'/hor_twt_hugin_fm_top.dat', names=['cdp_x', 'cdp_y', 'twt'], sep='\s+')
hrz.head()

# Map horizon to seismic survey
hrz_mapped = cube.seis.surface_from_points(hrz, "twt", right=("cdp_x", "cdp_y"))

  0%|          | 0.00/12.3k [00:00<?, ? traces/s]

Loading as 3D
Fast direction is TRACE_SEQUENCE_FILE


Converting SEGY:   0%|          | 0.00/12.3k [00:00<?, ? traces/s]

<xarray.Dataset>
Dimensions:  (iline: 61, xline: 202)
Coordinates:
  * iline    (iline) uint16 10090 10091 10092 10093 ... 10147 10148 10149 10150
  * xline    (xline) uint16 2150 2151 2152 2153 2154 ... 2348 2349 2350 2351
    twt      (iline, xline) float64 2.741e+03 2.742e+03 ... 2.632e+03 2.635e+03
    cdp_x    (iline, xline) float32 4.364e+05 4.364e+05 ... 4.342e+05 4.341e+05
    cdp_y    (iline, xline) float32 6.477e+06 6.477e+06 ... 6.479e+06 6.479e+06
Data variables:
    *empty*
Attributes: (12/13)
    ns:                  None
    sample_rate:         4.0
    text:                C 1 SEGY OUTPUT FROM Petrel 2017.2 Saturday, June 06...
    measurement_system:  m
    d3_domain:           None
    epsg:                None
    ...                  ...
    corner_points_xy:    None
    source_file:         volve10r12-full-twt-sub3d.sgy
    srd:                 None
    datatype:            None
    percentiles:         [-6.595060190682801, -6.114936243337226, -1.50399996...
    coord_scalar:        -100.0

In [8]:
cube

<xarray.Dataset>
Dimensions:  (iline: 61, xline: 202, twt: 850)
Coordinates:
  * iline    (iline) uint16 10090 10091 10092 10093 ... 10147 10148 10149 10150
  * xline    (xline) uint16 2150 2151 2152 2153 2154 ... 2348 2349 2350 2351
  * twt      (twt) float64 4.0 8.0 12.0 16.0 ... 3.392e+03 3.396e+03 3.4e+03
    cdp_x    (iline, xline) float32 4.364e+05 4.364e+05 ... 4.342e+05 4.341e+05
    cdp_y    (iline, xline) float32 6.477e+06 6.477e+06 ... 6.479e+06 6.479e+06
Data variables:
    data     (iline, xline, twt) float32 0.02057 0.02204 0.01966 ... 0.0 0.0 0.0
Attributes: (12/13)
    ns:                  None
    sample_rate:         4.0
    text:                C 1 SEGY OUTPUT FROM Petrel 2017.2 Saturday, June 06...
    measurement_system:  m
    d3_domain:           None
    epsg:                None
    ...                  ...
    corner_points_xy:    None
    source_file:         volve10r12-full-twt-sub3d.sgy
    srd:                 None
    datatype:            None
    percentiles:         [-6.595060190682801, -6.114936243337226, -1.50399996...
    coord_scalar:        -100.0

In [9]:
hrz_mapped

<xarray.Dataset>
Dimensions:  (iline: 61, xline: 202)
Coordinates:
  * iline    (iline) uint16 10090 10091 10092 10093 ... 10147 10148 10149 10150
  * xline    (xline) uint16 2150 2151 2152 2153 2154 ... 2348 2349 2350 2351
    twt      (iline, xline) float64 2.741e+03 2.742e+03 ... 2.632e+03 2.635e+03
    cdp_x    (iline, xline) float32 4.364e+05 4.364e+05 ... 4.342e+05 4.341e+05
    cdp_y    (iline, xline) float32 6.477e+06 6.477e+06 ... 6.479e+06 6.479e+06
Data variables:
    *empty*
Attributes: (12/13)
    ns:                  None
    sample_rate:         4.0
    text:                C 1 SEGY OUTPUT FROM Petrel 2017.2 Saturday, June 06...
    measurement_system:  m
    d3_domain:           None
    epsg:                None
    ...                  ...
    corner_points_xy:    None
    source_file:         volve10r12-full-twt-sub3d.sgy
    srd:                 None
    datatype:            None
    percentiles:         [-6.595060190682801, -6.114936243337226, -1.50399996...
    coord_scalar:        -100.0

# Interactive plot using ipywidgets

In [5]:
import ipywidgets as widgets
from ipywidgets import interactive


opt = dict(
    x="xline",
    y="twt",
    add_colorbar=True,
    interpolation="spline16",
    robust=True,
    yincrease=False,
    cmap="RdBu",
    vmin=-5,
    vmax=5,
    alpha = 0.8
)


def f(inline_val):
    f, ax = plt.subplots(1,2,figsize=(20,5))

    ####### map plot here ######

    tform = cube.seis.get_affine_transform()
    axs = ax[0]
    mesh = axs.pcolormesh(
        hrz_mapped.iline,
        hrz_mapped.xline,
        hrz_mapped.twt.T,
        shading="auto",
        transform=tform + axs.transData,
        cmap = 'viridis_r'
    )
    axs.set_aspect(1)
    axs.axis('off')
    dx = 5 # number of lines in x and y grid
    line_ext = 5 #extension of grid lines
    text_ext = 5

    axs.plot(corners[:,0], corners[:,1], lw = 4, c='white')
    axs.plot([corners[0,0]  ,corners[-1,0]], [corners[0,1]  ,corners[-1,1]],  lw = 4, c='white')
    #Grid 
    seis_grid_x = np.round(np.linspace(cube.iline.min(),cube.iline.max(), dx).astype('int64'),-1)
    seis_grid_y = np.round(np.linspace(cube.xline.min(),cube.xline.max(), dx).astype('int64'),-1)

    props = dict( facecolor='white',edgecolor = 'k', alpha=1)

    _ = axs.plot([inline_val, inline_val], [cube.xline[0], cube.xline[-1]], transform=tform + axs.transData, color="k", lw = 2)
    axs.text(inline_val-5, cube.xline[-1]+6, transform=tform + axs.transData, s=str(inline_val), rotation= 75, fontsize = 8, bbox = props)

    #Grid loop:
    for i in range(seis_grid_x.shape[0]):
        # Inline grid
        _ = axs.plot([cube.iline[0]-line_ext, cube.iline[-1]+line_ext], [seis_grid_y[i], seis_grid_y[i]], transform=tform + axs.transData, color="grey", lw = .5)
        #Crossline grid
        _ = axs.plot([seis_grid_x[i], seis_grid_x[i]], [cube.xline[0]-line_ext, cube.xline[-1]+line_ext], transform=tform + axs.transData, color="grey", lw = .5)
        #Inline numbers
        #axs.text(seis_grid_x[i], cube.xline[-1]+10, transform=tform + axs.transData, s=str(seis_grid_x[i]), rotation= 75, fontsize = 8)
        #Crossline numbers
        axs.text(cube.iline[-1]+text_ext, seis_grid_y[i]+5, transform=tform + axs.transData, s=str(seis_grid_y[i]), rotation= 75 - 90, fontsize = 10)

    #'Inline #' label
    axs.text(seis_grid_x[len(seis_grid_x)//2]-5, cube.xline[-1]+20, transform=tform + axs.transData, s='Inline #', rotation= 75  , fontsize = 10);
    #'Crossline #' label
    axs.text(cube.iline[-1]+10, seis_grid_y[len(seis_grid_y)//2]+8, transform=tform + axs.transData, s='Crossline #', rotation= 75 - 90  , fontsize = 10);
    

    ####### vertical seismic plot here ######

    cube.data.sel(iline=inline_val, twt=slice(2300, 3000)).plot.imshow(ax=ax[1],   **opt )
    x, t = hrz_mapped.sel(iline=inline_val).xline, hrz_mapped.sel(iline=inline_val).twt
    ax[1].plot(x, t, color="k", lw = 3)
    ax[1].invert_xaxis()
    ax[1].set_xlabel('Crossline #')
    ax[1].set_ylabel('TWT (ms)')
    ax[1].set_title('Inline #'+str(inline_val))
    ax[1].text(x[11],t[0], s = 'Top Hugin', bbox = props, fontsize = 6)

    plt.show()
    
interactive_plot = interactive(f, inline_val=(cube.iline.data.min(), cube.iline.data.max()))
output = interactive_plot.children[-1]
output.layout.height = '480px'
interactive_plot

interactive(children=(IntSlider(value=10120, description='inline_val', max=10150, min=10090), Output(layout=La…

# Creating a video using imageio

In [6]:
filenames = []


opt = dict(
    x="xline",
    y="twt",
    add_colorbar=True,
    interpolation="spline16",
    robust=True,
    yincrease=False,
    cmap="RdBu",
    vmin=-5,
    vmax=5,
    alpha = 0.8
)

for inline_val in cube.iline.data:
    f, ax = plt.subplots(1,2,figsize=(20,5))
    tform = cube.seis.get_affine_transform()
    axs = ax[0]
    mesh = axs.pcolormesh(
        hrz_mapped.iline,
        hrz_mapped.xline,
        hrz_mapped.twt.T,
        shading="auto",
        transform=tform + axs.transData,
        cmap = 'viridis_r'
    )
    axs.set_aspect(1)
    axs.axis('off')
    dx = 5 # number of lines in x and y grid
    line_ext = 5 #extension of grid lines
    text_ext = 5

    axs.plot(corners[:,0], corners[:,1], lw = 4, c='white')
    axs.plot([corners[0,0]  ,corners[-1,0]], [corners[0,1]  ,corners[-1,1]],  lw = 4, c='white')

    seis_grid_x = np.round(np.linspace(cube.iline.min(),cube.iline.max(), dx).astype('int64'),-1)
    seis_grid_y = np.round(np.linspace(cube.xline.min(),cube.xline.max(), dx).astype('int64'),-1)

    props = dict( facecolor='white',edgecolor = 'k', alpha=1)

    _ = axs.plot([inline_val, inline_val], [cube.xline[0], cube.xline[-1]], transform=tform + axs.transData, color="k", lw = 2)
    axs.text(inline_val-5, cube.xline[-1]+6, transform=tform + axs.transData, s=str(inline_val), rotation= 75, fontsize = 8, bbox = props)

    #Grid loop:
    for i in range(seis_grid_x.shape[0]):
        # Inline grid
        _ = axs.plot([cube.iline[0]-line_ext, cube.iline[-1]+line_ext], [seis_grid_y[i], seis_grid_y[i]], transform=tform + axs.transData, color="grey", lw = .5)
        #Crossline grid
        _ = axs.plot([seis_grid_x[i], seis_grid_x[i]], [cube.xline[0]-line_ext, cube.xline[-1]+line_ext], transform=tform + axs.transData, color="grey", lw = .5)
        #Inline numbers
        #axs.text(seis_grid_x[i], cube.xline[-1]+10, transform=tform + axs.transData, s=str(seis_grid_x[i]), rotation= 75, fontsize = 8)
        #Crossline numbers
        axs.text(cube.iline[-1]+text_ext, seis_grid_y[i]+5, transform=tform + axs.transData, s=str(seis_grid_y[i]), rotation= 75 - 90, fontsize = 10)

    #'Inline #' label
    axs.text(seis_grid_x[len(seis_grid_x)//2]-5, cube.xline[-1]+20, transform=tform + axs.transData, s='Inline #', rotation= 75  , fontsize = 10);
    axs.text(cube.iline[-1]+10, seis_grid_y[len(seis_grid_y)//2]+8, transform=tform + axs.transData, s='Crossline #', rotation= 75 - 90  , fontsize = 10);
    ### Vertical seismic section ###


    cube.data.sel(iline=inline_val, twt=slice(2300, 3000)).plot.imshow(ax=ax[1],   **opt )
    x, t = hrz_mapped.sel(iline=inline_val).xline, hrz_mapped.sel(iline=inline_val).twt
    ax[1].plot(x, t, color="k", lw = 3)
    ax[1].invert_xaxis()
    ax[1].set_xlabel('Crossline #')
    ax[1].set_ylabel('TWT (ms)')
    ax[1].set_title('Inline #'+str(inline_val))
    ax[1].text(x[11],t[0], s = 'Top Hugin', bbox = props, fontsize = 6)
    filename = f'{inline_val}.png'
    filenames.append(filename)
    
    # save frame 
    plt.savefig(filename,format='png', bbox_inches='tight', dpi=150)
    plt.close()


# Create video from .png files in folder
with imageio.get_writer('volve10r12-full-twt-sub3d.mp4', mode='I', fps = 10) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Remove .png files in folder
for filename in set(filenames):
    os.remove(filename)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (2299, 701) to (2304, 704) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


# References
Segysak - documentation (https://segysak.readthedocs.io/en/latest/index.html) , github: https://github.com/trhallam/segysak

The author thanks Equinor AS, the former Volve license partners ExxonMobil Exploration and Production Norway AS and Bayerngas (now Spirit Energy) for permission to use the Volve dataset, and to the many persons who have contributed to the work here. Please visit data.equinor.com for more information about the Volve dataset and license terms of use